#Multi Chromosome, tiap chromosome hanya diisi angka non-given

In [ ]:
import time
import numpy as np
import random
import itertools

easy1_sudoku = np.array([
    [0, 4, 0, 0, 3, 8, 0, 0, 7],
    [0, 7, 9, 0, 1, 0, 0, 0, 8],
    [0, 5, 8, 0, 0, 0, 0, 2, 0],
    [0, 6, 4, 0, 0, 5, 0, 0, 3],
    [0, 0, 7, 3, 0, 4, 8, 0, 0],
    [2, 0, 0, 7, 0, 0, 5, 6, 0],
    [0, 2, 0, 0, 0, 0, 7, 3, 0],
    [7, 0, 0, 0, 5, 0, 6, 4, 0],
    [4, 0, 0, 1, 9, 0, 0, 8, 5]
])

medium1_sudoku = np.array([
    [6, 0, 0, 0, 3, 0, 0, 0, 1],
    [0, 1, 0, 6, 9, 0, 2, 8, 0],
    [5, 0, 9, 0, 0, 0, 0, 0, 0],
    [0, 6, 2, 0, 8, 3, 0, 0, 0],
    [7, 0, 0, 0, 0, 0, 0, 0, 4],
    [0, 0, 0, 2, 7, 0, 3, 1, 0],
    [0, 0, 0, 0, 0, 0, 5, 0, 2],
    [0, 5, 4, 0, 6, 7, 0, 9, 0],
    [9, 0, 0, 0, 5, 0, 0, 0, 8]
])

hard1_sudoku = np.array([
    [9, 5, 6, 3, 0, 1, 8, 0, 0],
    [0, 0, 0, 0, 4, 0, 2, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 9],
    [0, 6, 0, 4, 0, 0, 5, 0, 0],
    [4, 0, 0, 0, 6, 0, 0, 0, 7],
    [0, 0, 1, 0, 0, 2, 0, 6, 0],
    [8, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 7, 0, 9, 0, 0, 0, 0],
    [0, 0, 2, 7, 0, 4, 9, 3, 6],
])

# ------------------------------
# Pretty print Sudoku grid
# ------------------------------
def print_sudoku(grid, title=None):
    if title:
        print("\n" + "=" * 40)
        print(title)
        print("=" * 40)
    for i, row in enumerate(grid):
        row_str = ""
        for j, val in enumerate(row):
            char = "." if val == 0 else str(val)
            sep = " "
            if j in [2, 5]:
                sep = " | "
            row_str += char + sep
        print(row_str)
        if i in [2, 5]:
            print("-" * 21)
    print("=" * 40)

# ------------------------------
# Precompute candidate map (dari GIVENS saja)
# ------------------------------
def precompute_candidate_map(base_grid):
    fixed = base_grid != 0
    cand = {}
    for r in range(9):
        row_used = set(base_grid[r, fixed[r, :]])
        for c in range(9):
            if fixed[r, c]:
                continue
            col_used = set(base_grid[fixed[:, c], c])
            br, bc = (r // 3) * 3, (c // 3) * 3
            block = base_grid[br:br+3, bc:bc+3]
            block_fixed = fixed[br:br+3, bc:bc+3]
            block_used = set(block[block_fixed])
            used = row_used | col_used | block_used
            cand[(r, c)] = set(range(1, 10)) - used
    return cand, fixed

# ------------------------------
# Posisi non-given per blok 3x3
# ------------------------------
def compute_block_positions(base_grid, fixed):
    block_pos = {}
    for bx in range(3):
        for by in range(3):
            rows = range(bx*3, (bx+1)*3)
            cols = range(by*3, (by+1)*3)
            positions = []
            for r in rows:
                for c in cols:
                    if not fixed[r, c]:
                        positions.append((r, c))
            block_pos[(bx, by)] = positions
    return block_pos

# ------------------------------
# Individual: hanya angka non-given per blok
# ------------------------------
def create_individual(base_grid, fixed, block_pos):
    ind = {}
    for (bx, by), positions in block_pos.items():
        br, bc = bx*3, by*3
        block = base_grid[br:br+3, bc:bc+3]
        block_fixed_mask = fixed[br:br+3, bc:bc+3]
        used = set(block[block_fixed_mask])
        missing = list(set(range(1, 10)) - used)         # pastikan blok valid (1..9)
        random.shuffle(missing)
        # len(missing) == jumlah sel non-given pada blok
        ind[(bx, by)] = missing[:len(positions)]
    return ind

# ------------------------------
# Decode: isi non-given ke grid copy
# ------------------------------
def decode_to_grid(base_grid, individual, block_pos):
    grid = base_grid.copy()
    for (bx, by), positions in block_pos.items():
        vals = individual[(bx, by)]
        for i, (r, c) in enumerate(positions):
            grid[r, c] = vals[i]
    return grid

# ------------------------------
# Fitness: duplikasi baris + kolom
# (blok valid by construction)
# ------------------------------
def fitness_individual(individual, base_grid, block_pos):
    # buat set berisi nilai unik per baris dan kolom
    row_vals = [set() for _ in range(9)]
    col_vals = [set() for _ in range(9)]

    # tambahkan semua angka 'given' dulu
    fixed = base_grid != 0
    for r in range(9):
        for c in range(9):
            if fixed[r, c]:
                val = base_grid[r, c]
                row_vals[r].add(val)
                col_vals[c].add(val)

    # tambahkan semua angka dari individu
    for (bx, by), positions in block_pos.items():
        vals = individual[(bx, by)]
        for i, (r, c) in enumerate(positions):
            v = vals[i]
            row_vals[r].add(v)
            col_vals[c].add(v)

    # hitung penalti duplikasi
    score = 0
    for i in range(9):
        score += (9 - len(row_vals[i]))  # baris
        score += (9 - len(col_vals[i]))  # kolom

    return score


# ------------------------------
# Tournament selection
# ------------------------------
def tournament_selection(population, base_grid, block_pos, k=3):
    selected = random.sample(population, k)
    selected.sort(key=lambda ind: fitness_individual(ind, base_grid, block_pos))
    return selected[0]


# ------------------------------
# Crossover: block-wise
# ------------------------------

def crossover1(p1, p2, base_grid, block_pos):
    child1 = {}
    child2 = {}

    for bx in range(3):
        for by in range(3):
            key = (bx, by)

            if random.random() < 0.5:
                # child1 ambil dari p1, child2 dari p2
                child1[key] = list(p1[key])
                child2[key] = list(p2[key])
            else:
                # child1 ambil dari p2, child2 dari p1
                child1[key] = list(p2[key])
                child2[key] = list(p1[key])

    return child1, child2


def crossover2(p1, p2, base_grid, block_pos):
    """
    Crossover per-blok dengan mempertimbangkan seberapa banyak
    angka unik di baris dan kolom yang dihasilkan.
    Child1 dibangun greedy, Child2 adalah komplemennya.
    """
    fixed = base_grid != 0

    # Mulai dari nilai-nilai given
    row_vals = [set(base_grid[r, fixed[r, :]]) for r in range(9)]
    col_vals = [set(base_grid[fixed[:, c], c]) for c in range(9)]

    child1 = {}
    child2 = {}

    # Urutan blok diacak supaya tidak bias
    block_keys = list(block_pos.keys())
    random.shuffle(block_keys)

    def score_after_block(row_sets, col_sets, positions, vals):
        # copy ringan
        tmp_rows = [set(s) for s in row_sets]
        tmp_cols = [set(s) for s in col_sets]
        for i, (r, c) in enumerate(positions):
            v = vals[i]
            tmp_rows[r].add(v)
            tmp_cols[c].add(v)
        score = 0
        for r in range(9):
            score += len(tmp_rows[r])
        for c in range(9):
            score += len(tmp_cols[c])
        return score

    for key in block_keys:
        positions = block_pos[key]
        vals1 = p1[key]
        vals2 = p2[key]

        # hitung skor jika blok ini diambil dari p1 atau p2
        s1 = score_after_block(row_vals, col_vals, positions, vals1)
        s2 = score_after_block(row_vals, col_vals, positions, vals2)

        if s1 >= s2:
            chosen1 = vals1
            chosen2 = vals2
        else:
            chosen1 = vals2
            chosen2 = vals1

        # isi child1 dengan blok yang "lebih bagus", child2 dengan komplemennya
        child1[key] = list(chosen1)
        child2[key] = list(chosen2)

        # update row_vals & col_vals sesuai blok yang dipakai child1
        for i, (r, c) in enumerate(positions):
            v = chosen1[i]
            row_vals[r].add(v)
            col_vals[c].add(v)

    return child1, child2



# ------------------------------
# Filtered mutation (pakai candidate_map)
# swap 2 sel non-given DI DALAM blok jika lolos filter
def mutate(individual, mutation_rate, candidate_map, block_pos):
    new_ind = {k: list(v) for k, v in individual.items()}
    for (bx, by), positions in block_pos.items():
        if len(positions) < 2:
            continue
        if random.random() >= mutation_rate:
            continue

        pairs = list(itertools.combinations(range(len(positions)), 2))
        random.shuffle(pairs)

        for i, j in pairs:
            (r1, c1) = positions[i]
            (r2, c2) = positions[j]
            v1 = new_ind[(bx, by)][i]
            v2 = new_ind[(bx, by)][j]
            cand1 = candidate_map.get((r1, c1), set())
            cand2 = candidate_map.get((r2, c2), set())
            if (v1 in cand2) or (v2 in cand1):
                new_ind[(bx, by)][i], new_ind[(bx, by)][j] = v2, v1
                break
    return new_ind

# ------------------------------
# GA main loop
# ------------------------------
def genetic_sudoku_solver(base_grid, crossover_func):
    candidate_map, fixed = precompute_candidate_map(base_grid)
    block_pos = compute_block_positions(base_grid, fixed)
    population = [create_individual(base_grid, fixed, block_pos) for _ in range(POP_SIZE)]

    # contoh individu awal
    sample_ind = population[0]
    print("\n===== Contoh Individu Awal (GENOTYPE) =====")
    for key, val in sample_ind.items():
        print(f"Blok {key}: {val}")

    sample_grid = decode_to_grid(base_grid, sample_ind, block_pos)
    print_sudoku(sample_grid, "Contoh Individu Awal (PHENOTYPE)")

    print("\nMulai evolusi...")
    total_start = time.time()

    # === parameter mating pool & elitism ===
    mating_fraction = MATING_SIZE
    mating_size = int(POP_SIZE * mating_fraction)
    elite_size = POP_SIZE - mating_size
    for gen in range(MAX_GENERATIONS):
        gen_start = time.time()

        # sort populasi berdasarkan fitness (kecil = bagus)
        population.sort(key=lambda ind: fitness_individual(ind, base_grid, block_pos))
        best = population[0]
        best_fit = fitness_individual(best, base_grid, block_pos)

        gen_time = time.time() - gen_start
        if gen % 10 == 0:
            print(f"Gen {gen:4d} | Fitness terbaik = {best_fit:2d} | Waktu per generasi = {gen_time:.4f} detik")

        if best_fit == 0:
            total_time = time.time() - total_start
            print(f"\n✅ Solusi ditemukan di generasi {gen} fitness: {best_fit}")
            print(f"Total waktu eksekusi: {total_time:.4f} detik")
            return decode_to_grid(base_grid, best, block_pos)

        # === 1) ELITISM: ambil individu terbaik sejumlah elite_size ===
        elites = [
            {k: list(v) for k, v in population[i].items()}
            for i in range(elite_size)
        ]

        # === 2) SELECTION: bangun mating pool dengan tournament ===
        mating_pool = [
            tournament_selection(population, base_grid, block_pos)
            for _ in range(POP_SIZE)
        ]


        # === 3) CROSSOVER + MUTATION: pairing acak lewat shuffle ===
        children = []

        while len(children) < mating_size:
            p1, p2 = random.sample(mating_pool, 2)

            # Buat 1 child dari pasangan (p1, p2)
            if random.random() < CROSSOVER_RATE_PARENT:
                c1, c2 = crossover_func(p1, p2, base_grid, block_pos)
            else:
                better = p1 if fitness_individual(p1, base_grid, block_pos) < fitness_individual(p2, base_grid, block_pos) else p2
                worse  = p2 if better is p1 else p1
                # dua anak = copy dari better & worse
                c1 = {k: list(v) for k, v in better.items()}
                c2 = {k: list(v) for k, v in worse.items()}

            c1 = mutate(c1, MUTATION_RATE, candidate_map, block_pos)
            c2 = mutate(c2, MUTATION_RATE, candidate_map, block_pos)
            children.append(c1)

            if len(children) < mating_size:
              children.append(c2)


        # === 4) Populasi baru = elit + anak hasil mating ===
        population = elites + children


    total_time = time.time() - total_start
    print("\n❌ Tidak ditemukan solusi dalam batas generasi.")
    print(f"Total waktu eksekusi: {total_time:.4f} detik")
    return decode_to_grid(base_grid, best, block_pos)


# ------------------------------
# Jalankan
# ------------------------------
# print_sudoku(sudoku, "Puzzle Awal")
# solution = genetic_sudoku_solver(sudoku)
# print_sudoku(solution, "Solusi Akhir (GA)")

In [ ]:
POP_SIZE = 100
MUTATION_RATE = 0.2
CROSSOVER_RATE_PARENT = 0.9
TOURNAMENT_SIZE = 3
MATING_SIZE = 0.95
MAX_GENERATIONS = 60000


In [ ]:
SEED = 1245      # ubah angka ini untuk eksperimen lain
SUDOKU = easy1_sudoku

In [ ]:
def run_experiment(seed, crossover_func, sudoku_grid, label=""):
    random.seed(seed)
    np.random.seed(seed)

    print(f"\n=== Eksperimen Seed {seed} | Puzzle {label or 'Custom'} | "
          f"Crossover = {crossover_func.__name__} ===")

    solution = genetic_sudoku_solver(sudoku_grid, crossover_func=crossover_func)
    print_sudoku(solution, f"Solusi Akhir (Seed {seed} - {label or 'Custom'})")


##Crossover subblock wise (menukar subblock dari 2 parent scr random (50%))

In [ ]:
block_pos = compute_block_positions(SUDOKU, SUDOKU != 0)
run_experiment(
    SEED,
    crossover1,
    SUDOKU
)



=== Eksperimen Seed 1245 | Puzzle Custom | Crossover = crossover1 ===

===== Contoh Individu Awal (GENOTYPE) =====
Blok (0, 0): [6, 3, 1, 2]
Blok (0, 1): [4, 7, 2, 6, 9, 5]
Blok (0, 2): [1, 9, 4, 6, 3, 5]
Blok (1, 0): [1, 8, 5, 3, 9]
Blok (1, 1): [8, 2, 1, 6, 9]
Blok (1, 2): [1, 2, 4, 7, 9]
Blok (2, 0): [3, 5, 1, 8, 6, 9]
Blok (2, 1): [4, 8, 6, 7, 2, 3]
Blok (2, 2): [1, 2, 9]

Contoh Individu Awal (PHENOTYPE)
6 4 3 | 4 3 8 | 1 9 7 
1 7 9 | 7 1 2 | 4 6 8 
2 5 8 | 6 9 5 | 3 2 5 
---------------------
1 6 4 | 8 2 5 | 1 2 3 
8 5 7 | 3 1 4 | 8 4 7 
2 3 9 | 7 6 9 | 5 6 9 
---------------------
3 2 5 | 4 8 6 | 7 3 1 
7 1 8 | 7 5 2 | 6 4 2 
4 6 9 | 1 9 3 | 9 8 5 

Mulai evolusi...
Gen    0 | Fitness terbaik = 29 | Waktu per generasi = 0.0068 detik
Gen   10 | Fitness terbaik = 15 | Waktu per generasi = 0.0047 detik
Gen   20 | Fitness terbaik = 10 | Waktu per generasi = 0.0039 detik
Gen   30 | Fitness terbaik =  6 | Waktu per generasi = 0.0045 detik
Gen   40 | Fitness terbaik =  6 | Waktu per g

In [ ]:
block_pos = compute_block_positions(SUDOKU, SUDOKU != 0)
run_experiment(
    SEED,
    crossover1,
    medium1_sudoku
)


Streaming output truncated to the last 5000 lines.
Gen 10190 | Fitness terbaik =  4 | Waktu per generasi = 0.0034 detik
Gen 10200 | Fitness terbaik =  4 | Waktu per generasi = 0.0033 detik
Gen 10210 | Fitness terbaik =  4 | Waktu per generasi = 0.0033 detik
Gen 10220 | Fitness terbaik =  4 | Waktu per generasi = 0.0033 detik
Gen 10230 | Fitness terbaik =  4 | Waktu per generasi = 0.0040 detik
Gen 10240 | Fitness terbaik =  4 | Waktu per generasi = 0.0032 detik
Gen 10250 | Fitness terbaik =  4 | Waktu per generasi = 0.0033 detik
Gen 10260 | Fitness terbaik =  4 | Waktu per generasi = 0.0033 detik
Gen 10270 | Fitness terbaik =  4 | Waktu per generasi = 0.0035 detik
Gen 10280 | Fitness terbaik =  4 | Waktu per generasi = 0.0040 detik
Gen 10290 | Fitness terbaik =  4 | Waktu per generasi = 0.0034 detik
Gen 10300 | Fitness terbaik =  4 | Waktu per generasi = 0.0036 detik
Gen 10310 | Fitness terbaik =  4 | Waktu per generasi = 0.0033 detik
Gen 10320 | Fitness terbaik =  4 | Waktu per generas

In [ ]:
block_pos = compute_block_positions(SUDOKU, SUDOKU != 0)
run_experiment(
    SEED,
    crossover1,
    hard1_sudoku
)


Streaming output truncated to the last 5000 lines.
Gen 10190 | Fitness terbaik =  4 | Waktu per generasi = 0.0034 detik
Gen 10200 | Fitness terbaik =  4 | Waktu per generasi = 0.0032 detik
Gen 10210 | Fitness terbaik =  4 | Waktu per generasi = 0.0032 detik
Gen 10220 | Fitness terbaik =  4 | Waktu per generasi = 0.0032 detik
Gen 10230 | Fitness terbaik =  4 | Waktu per generasi = 0.0034 detik
Gen 10240 | Fitness terbaik =  4 | Waktu per generasi = 0.0039 detik
Gen 10250 | Fitness terbaik =  4 | Waktu per generasi = 0.0032 detik
Gen 10260 | Fitness terbaik =  4 | Waktu per generasi = 0.0033 detik
Gen 10270 | Fitness terbaik =  4 | Waktu per generasi = 0.0032 detik
Gen 10280 | Fitness terbaik =  4 | Waktu per generasi = 0.0044 detik
Gen 10290 | Fitness terbaik =  4 | Waktu per generasi = 0.0032 detik
Gen 10300 | Fitness terbaik =  4 | Waktu per generasi = 0.0032 detik
Gen 10310 | Fitness terbaik =  4 | Waktu per generasi = 0.0043 detik
Gen 10320 | Fitness terbaik =  4 | Waktu per generas

##Crossover dengan Heuristic Block-Based Crossover

In [ ]:
block_pos = compute_block_positions(SUDOKU, SUDOKU != 0)
run_experiment(
    SEED,
    crossover2,
    SUDOKU
)



=== Eksperimen Seed 1245 | Puzzle Custom | Crossover = crossover2 ===

===== Contoh Individu Awal (GENOTYPE) =====
Blok (0, 0): [6, 3, 1, 2]
Blok (0, 1): [4, 7, 2, 6, 9, 5]
Blok (0, 2): [1, 9, 4, 6, 3, 5]
Blok (1, 0): [1, 8, 5, 3, 9]
Blok (1, 1): [8, 2, 1, 6, 9]
Blok (1, 2): [1, 2, 4, 7, 9]
Blok (2, 0): [3, 5, 1, 8, 6, 9]
Blok (2, 1): [4, 8, 6, 7, 2, 3]
Blok (2, 2): [1, 2, 9]

Contoh Individu Awal (PHENOTYPE)
6 4 3 | 4 3 8 | 1 9 7 
1 7 9 | 7 1 2 | 4 6 8 
2 5 8 | 6 9 5 | 3 2 5 
---------------------
1 6 4 | 8 2 5 | 1 2 3 
8 5 7 | 3 1 4 | 8 4 7 
2 3 9 | 7 6 9 | 5 6 9 
---------------------
3 2 5 | 4 8 6 | 7 3 1 
7 1 8 | 7 5 2 | 6 4 2 
4 6 9 | 1 9 3 | 9 8 5 

Mulai evolusi...
Gen    0 | Fitness terbaik = 29 | Waktu per generasi = 0.0040 detik
Gen   10 | Fitness terbaik =  9 | Waktu per generasi = 0.0043 detik
Gen   20 | Fitness terbaik =  4 | Waktu per generasi = 0.0040 detik

✅ Solusi ditemukan di generasi 23 fitness: 0
Total waktu eksekusi: 0.7221 detik

Solusi Akhir (Seed 1245 - Custo

In [ ]:
block_pos = compute_block_positions(SUDOKU, SUDOKU != 0)
run_experiment(
    SEED,
    crossover2,
    medium1_sudoku
)



=== Eksperimen Seed 1245 | Puzzle Custom | Crossover = crossover2 ===

===== Contoh Individu Awal (GENOTYPE) =====
Blok (0, 0): [7, 4, 2, 3, 8]
Blok (0, 1): [2, 7, 1, 5, 8, 4]
Blok (0, 2): [3, 9, 5, 7, 4, 6]
Blok (1, 0): [1, 9, 4, 5, 3, 8]
Blok (1, 1): [1, 9, 5, 6, 4]
Blok (1, 2): [9, 2, 5, 7, 6, 8]
Blok (2, 0): [6, 3, 7, 2, 1, 8]
Blok (2, 1): [9, 1, 8, 3, 4, 2]
Blok (2, 2): [1, 4, 7, 6, 3]

Contoh Individu Awal (PHENOTYPE)
6 7 4 | 2 3 7 | 3 9 1 
2 1 3 | 6 9 1 | 2 8 5 
5 8 9 | 5 8 4 | 7 4 6 
---------------------
1 6 2 | 1 8 3 | 9 2 5 
7 9 4 | 9 5 6 | 7 6 4 
5 3 8 | 2 7 4 | 3 1 8 
---------------------
6 3 7 | 9 1 8 | 5 1 2 
2 5 4 | 3 6 7 | 4 9 7 
9 1 8 | 4 5 2 | 6 3 8 

Mulai evolusi...
Gen    0 | Fitness terbaik = 35 | Waktu per generasi = 0.0043 detik
Gen   10 | Fitness terbaik = 12 | Waktu per generasi = 0.0034 detik
Gen   20 | Fitness terbaik =  6 | Waktu per generasi = 0.0039 detik
Gen   30 | Fitness terbaik =  5 | Waktu per generasi = 0.0038 detik
Gen   40 | Fitness terbaik =  

In [ ]:

block_pos = compute_block_positions(SUDOKU, SUDOKU != 0)
run_experiment(
    SEED,
    crossover2,
    hard1_sudoku
)



=== Eksperimen Seed 1245 | Puzzle Custom | Crossover = crossover2 ===

===== Contoh Individu Awal (GENOTYPE) =====
Blok (0, 0): [4, 3, 1, 2, 8, 7]
Blok (0, 1): [5, 8, 2, 7, 9, 6]
Blok (0, 2): [1, 7, 4, 6, 3, 5]
Blok (1, 0): [2, 9, 5, 7, 3, 8]
Blok (1, 1): [1, 9, 5, 8, 7, 3]
Blok (1, 2): [2, 3, 1, 8, 4, 9]
Blok (2, 0): [4, 9, 5, 6, 1, 3]
Blok (2, 1): [6, 5, 1, 2, 3, 8]
Blok (2, 2): [1, 8, 2, 5, 7, 4]

Contoh Individu Awal (PHENOTYPE)
9 5 6 | 3 5 1 | 8 1 7 
4 3 1 | 8 4 2 | 2 4 6 
2 8 7 | 7 9 6 | 3 5 9 
---------------------
2 6 9 | 4 1 9 | 5 2 3 
4 5 7 | 5 6 8 | 1 8 7 
3 8 1 | 7 3 2 | 4 6 9 
---------------------
8 4 9 | 6 5 1 | 1 8 2 
5 6 7 | 2 9 3 | 5 7 4 
1 3 2 | 7 8 4 | 9 3 6 

Mulai evolusi...
Gen    0 | Fitness terbaik = 31 | Waktu per generasi = 0.0074 detik
Gen   10 | Fitness terbaik = 10 | Waktu per generasi = 0.0034 detik
Gen   20 | Fitness terbaik =  4 | Waktu per generasi = 0.0039 detik
Gen   30 | Fitness terbaik =  2 | Waktu per generasi = 0.0034 detik

✅ Solusi ditemukan d